# 모듈 import

In [1]:
import sys
import random
import pickle

import pandas as pd
import numpy as np
import json
from konlpy.tag import Twitter
from sklearn.externals import joblib
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB , GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [2]:
import json
import os
import random

import konlpy

#import elice_challenge as ec

# 파일 읽기

In [3]:
test_out_data = open('./data/test.output')
#test_out_data = test_out_data.read()
#3test_out_data = test_out_data.splitlines()

In [4]:
train_data = json.load(open('./data/train_data.json'))

test_data = json.load(open('./data/test.input.json'))

with open('./data/test.output') as fp:
    test_out = fp.read()
    test_out_data = test_out.splitlines()

grading_data = json.load(open('./data/grading.input.json'))

파일이 제대로 읽혔는지 확인합니다.

In [5]:
print(len(train_data))
print(len(test_data))
print(len(test_out_data))
print(len(grading_data))


76500
4500
4500
9000


In [6]:
test_out_data[:10]

['연합뉴스', '뉴시스', 'MBN', '동아일보', '채널A', '오마이뉴스', 'YTN', 'JTBC', '동아일보', '파이낸셜뉴스']

In [7]:
train_df = pd.DataFrame(train_data)[['text','writing']]
test_df = pd.DataFrame(test_data)[['text']]
test_df['writing'] = test_out_data
grading_df = pd.DataFrame(grading_data)[['text']]

In [8]:
train_df.head()

,text,writing
0,[앵커]청와대 민정수석을 지냈던 우병우씨의 소재를 놓고 설왕설래가 한창입니다. 지난...,JTBC
1,[<언론사명>]새누리당 원내대표 경선을 하루 앞둔 2일 청와대는 사실상 ‘불개입’ ...,서울신문
2,= 박근혜 대통령이 7일 오후 서울 은평구 한국여성정책연구원에서 열린 2016년 ...,뉴시스
3,이재오 의원. 사진=이재오 의원 페이스북 캡처[<언론사명> ] 새누리당을 탈당한 이...,아시아경제
4,"]곽상도(사진)새누리당 대구 중남구 예비후보는 24일 보도자료를 통해 ""새누리당이 ...",헤럴드경제


In [9]:
test_df.head()

,text,writing
0,"日, 위안부지원금 송금완료…'소녀상 압박' 우려(cg)[<언론사명>tv 제공]\n\...",연합뉴스
1,"새누리 ""안보와 안전, 민생이 최우선"" 더민주 ""박 대통령이 먼저 변해야"" 국민의당...",뉴시스
2,"""사실과 다르다"" 국민의당, '김수민 리베이트 의혹' 정면 반박국민의당 김수민/사진...",MBN
3,[<언론사명>]본보-r&r; 총선의 해 여론조사“신인 선택” 31%〉“현역” 24%...,동아일보
4,"김종인 더민주 대표는 자신이 던진 담뱃불이 어떻게 번지는 지, 느긋하게 즐기는 모습...",채널A


In [10]:
grading_df.head()

,text
0,4당 체제에도 청문회·국감·지방자치 합의 제도적 성과 14대 국회도 3대 정치개혁입...
1,눈감은 부산고검장\n (부산=<언론사명>) = 문무일 부산고검장이 11일 부...
2,[앵커] \n일본 규수 구마모토 지진으로 대규모 사상자가 발생하면서 정부가 신속대응...
3,박근혜 대통령이 총선을 하루 앞두고 '새로운 국회론'을 강력히 설파했다. 핵심 민생...
4,차은택 구속기소…특검 체제까지 사실상 '8부 능선' 넘어차은택/사진=<언론사명>'비...


# vectorize

In [18]:
import pickle

with open('stopwords.txt', 'rb') as f:
    p_X = pickle.load()

EOFError: Ran out of input

In [13]:
open('stopwords.txt').read()

''

In [11]:
#np.random.seed(10)
# shuffle_indices = np.random.permutation(np.arange(len(train_df)))
# X = train_df.text[shuffle_indices[:67000]]
# y = train_df.writing[shuffle_indices[:67000]]

# test_X = train_df.text[shuffle_indices[67000:]]
# test_y = train_df.writing[shuffle_indices[67000:]]

X = train_df.text
y = train_df.writing

In [20]:
stw = ['은/Josa','는/Josa','이/Josa','가/Josa','에서/Josa','의/Josa','을/Josa']

In [46]:
stw = ['은/Josa','[/Punctuation',']/Punctuation','(/Punctuation','/Punctuation',
       ',/Punctuation','./Punctuation','?/Punctuation','!/Punctuation','!!/Punctuation','"/Punctuation',
      ';/Punctuation',':/Punctuation','//Punctuation','-/Punctuation','~/Punctuation','.../Punctuation',
      '에/Josa','는/Josa','이/Josa','가/Josa','를/Josa','께서/Josa','게/Josa','고/Josa','마다/Josa','이라고도/Josa',
      '보다는/Josa','로운/Josa','까지야/Josa','했지만/Josa','만/Josa']

In [47]:
# 처음엔 이걸로 # vectorize train using Tfid(bag of words)

twitter = Twitter()

def tokenize_pos(doc):
    return ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]

vectorizer =  TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1,2), stop_words=stw)

X = vectorizer.fit_transform(X)

AttributeError: lower not found

In [ ]:
# # vectorize train using Tfid(bag of words)

# twitter = Twitter()

# def tokenize_pos(doc):
#     return ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]

# vectorizer =  TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1,2),
#                               use_idf=False, smooth_idf=False,
#                               vocabulary = reduced_feature_list)

# X = vectorizer.fit_transform(X)

In [ ]:
# # sparse matrix test
# import random
# import scipy.sparse as sparse
# import scipy.io
# import numpy as np

# def save_sparse_matrix(filename, x):
#     x_coo = x.tocoo()
#     row = x_coo.row
#     col = x_coo.col
#     data = x_coo.data
#     shape = x_coo.shape
#     np.savez(filename, row=row, col=col, data=data, shape=shape)

# def load_sparse_matrix(filename):
#     y = np.load(filename)
#     z = sparse.coo_matrix((y['data'], (y['row'], y['col'])), shape=y['shape'])
#     return z

# N=20000
# x = sparse.lil_matrix( (N,N) )
# for i in xrange(N):
#     x[random.randint(0,N-1),random.randint(0,N-1)]=random.randint(1,100)

# save_sparse_matrix('/tmp/my_array',x)
# load_sparse_matrix('/tmp/my_array.npz').tolil()

### Fitting train data

In [608]:
import xgboost

/Users/yoo/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [56]:
# fitting train data with classifier

#model = SGDClassifier(alpha=1.9e-6, n_iter=19).fit(X, y)
model = LogisticRegression(C=10200).fit(X,y)
#model = xgboost.XGBClassifier().fit(X,y)
#model = MultinomialNB(alpha=5).fit(X,y)
#model = ExtraTreesClassifier(n_estimators=10).fit(X,y)

In [686]:
pos_list.pos.str.contains('Josa')==False

0    True
Name: pos, dtype: bool

In [710]:
feature_list = vectorizer.get_feature_names()
pos_list = pd.DataFrame(feature_list,columns=['pos'])
reduced_feature_list = pos_list[pos_list.pos.str.contains('Josa')==False]

reduced_feature_list.shape

(1941228, 1)

In [23]:
# predict test data

def testX_to_vec(X):
    feature_list = vectorizer.get_feature_names()

    test_vectorizer =  TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1,2), vocabulary = feature_list)
    X_test = test_vectorizer.fit_transform(X)
    
    return X_test

In [15]:
# # reduced feature) predict test data

# def testX_to_vec(X):
#     feature_list = vectorizer.get_feature_names()

#     test_vectorizer =  TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1,2), vocabulary = reduced_feature_list)
#     X_test = test_vectorizer.fit_transform(X)
    
#     return X_test

In [ ]:
test_X = test_df.text

In [ ]:
# logistic 10200, true, true, stw 2
rank_class = predict_rank(test_X)
mrr(rank_class, test_out_data)

In [57]:
#test to vec
test_X = testX_to_vec(test_X)

NotFittedError: TfidfVectorizer - Vocabulary wasn't fitted.

In [ ]:
#grid search logistic C 5

alpha=[100, 500, 1000, 5000, 9800, 20000, 1000000, 10000000]
for i,val in enumerate(alpha):
    model = LogisticRegression(C=val).fit(X,y)
    rank_class = predict_rank(test_X)
    print(val, mrr(rank_class, test_df.writing))

In [35]:
# reduced 
test_pred = model.predict(test_X)

# accuracy
print(accuracy_score(test_df.writing, test_pred))


print(classification_report(test_df.writing, test_pred))

0.900222222222
             precision    recall  f1-score   support

       JTBC       0.97      0.88      0.92       139
     KBS 뉴스       0.96      0.92      0.94       372
        MBN       0.96      0.95      0.96       437
       TV조선       0.99      0.91      0.95        90
        YTN       0.89      0.90      0.89       232
       경향신문       0.81      0.62      0.70       100
       국민일보       0.74      0.82      0.78       336
       노컷뉴스       0.97      0.89      0.93        35
        뉴시스       0.91      0.98      0.94       661
       동아일보       0.88      0.88      0.88       290
       매일경제       0.73      0.68      0.70       186
       서울신문       0.79      0.85      0.82       253
      아시아경제       0.92      0.94      0.93        48
       연합뉴스       1.00      0.99      0.99       487
     연합뉴스TV       0.94      0.96      0.95       191
      오마이뉴스       0.98      0.98      0.98       121
        채널A       1.00      0.98      0.99       122
     파이낸셜뉴스       0.85      0.

In [602]:
# Logistic C=10
test_pred = model.predict(test_X)

# accuracy
print(accuracy_score(test_df.writing, test_pred))


print(classification_report(test_df.writing, test_pred))

0.79
             precision    recall  f1-score   support

       JTBC       1.00      0.68      0.81       139
     KBS 뉴스       0.89      0.92      0.90       372
        MBN       0.95      0.92      0.94       437
       TV조선       0.99      0.83      0.90        90
        YTN       0.75      0.92      0.83       232
       경향신문       0.59      0.60      0.59       100
       국민일보       0.51      0.86      0.64       336
       노컷뉴스       1.00      0.71      0.83        35
        뉴시스       0.88      0.95      0.92       661
       동아일보       0.94      0.65      0.77       290
       매일경제       0.71      0.46      0.56       186
       서울신문       0.99      0.31      0.47       253
      아시아경제       0.94      0.33      0.49        48
       연합뉴스       1.00      0.69      0.82       487
     연합뉴스TV       0.93      0.93      0.93       191
      오마이뉴스       1.00      0.63      0.77       121
        채널A       0.91      0.99      0.95       122
     파이낸셜뉴스       0.51      0.92      0.

In [598]:
# NB
test_pred = model.predict(X_test)

# accuracy
print(accuracy_score(test_df.writing, test_pred))
# 0.8111

print(classification_report(test_df.writing, test_pred))

0.600222222222
             precision    recall  f1-score   support

       JTBC       0.88      0.58      0.70       139
     KBS 뉴스       0.66      0.59      0.62       372
        MBN       0.69      0.73      0.71       437
       TV조선       1.00      0.14      0.25        90
        YTN       0.70      0.79      0.74       232
       경향신문       0.75      0.03      0.06       100
       국민일보       0.41      0.46      0.43       336
       노컷뉴스       1.00      0.17      0.29        35
        뉴시스       0.62      0.81      0.70       661
       동아일보       0.50      0.76      0.61       290
       매일경제       0.45      0.27      0.34       186
       서울신문       0.51      0.38      0.44       253
      아시아경제       0.00      0.00      0.00        48
       연합뉴스       0.51      0.82      0.62       487
     연합뉴스TV       0.75      0.70      0.72       191
      오마이뉴스       0.86      0.66      0.75       121
        채널A       0.92      0.54      0.68       122
     파이낸셜뉴스       0.59      0.

/Users/yoo/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [573]:
# forest
test_pred = model.predict(X_test)

# accuracy
print(accuracy_score(test_df.writing, test_pred))
# 0.8111

print(classification_report(test_df.writing, test_pred))

0.582888888889
             precision    recall  f1-score   support

       JTBC       0.64      0.55      0.59       139
     KBS 뉴스       0.65      0.84      0.73       372
        MBN       0.67      0.73      0.70       437
       TV조선       0.71      0.49      0.58        90
        YTN       0.72      0.67      0.69       232
       경향신문       0.21      0.08      0.12       100
       국민일보       0.33      0.36      0.34       336
       노컷뉴스       1.00      0.63      0.77        35
        뉴시스       0.45      0.88      0.60       661
       동아일보       0.50      0.56      0.53       290
       매일경제       0.38      0.06      0.10       186
       서울신문       0.49      0.36      0.41       253
      아시아경제       0.89      0.17      0.28        48
       연합뉴스       0.79      0.68      0.73       487
     연합뉴스TV       0.84      0.65      0.74       191
      오마이뉴스       1.00      0.64      0.78       121
        채널A       0.84      0.70      0.76       122
     파이낸셜뉴스       0.74      0.

In [540]:
#logistic
test_pred = model.predict(X_test)

# accuracy
print(accuracy_score(test_df.writing, test_pred))
# 0.8111

print(classification_report(test_df.writing, test_pred))

0.811111111111
             precision    recall  f1-score   support

       JTBC       1.00      0.71      0.83       139
     KBS 뉴스       0.92      0.90      0.91       372
        MBN       0.95      0.93      0.94       437
       TV조선       1.00      0.86      0.92        90
        YTN       0.76      0.94      0.84       232
       경향신문       0.66      0.60      0.63       100
       국민일보       0.54      0.85      0.66       336
       노컷뉴스       1.00      0.74      0.85        35
        뉴시스       0.88      0.95      0.92       661
       동아일보       0.92      0.72      0.81       290
       매일경제       0.71      0.55      0.62       186
       서울신문       0.95      0.38      0.54       253
      아시아경제       1.00      0.35      0.52        48
       연합뉴스       1.00      0.75      0.86       487
     연합뉴스TV       0.93      0.94      0.93       191
      오마이뉴스       1.00      0.64      0.78       121
        채널A       0.95      0.99      0.97       122
     파이낸셜뉴스       0.55      0.

In [612]:
#xgboost
test_pred = model.predict(X_test)

# accuracy
print(accuracy_score(test_df.writing, test_pred))
# 0.8111

print(classification_report(test_df.writing, test_pred))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [26]:
def predict_rank(X_test):
    
    test_prob = model.predict_proba(X_test)

    test_class = pd.DataFrame((-test_prob).argsort())
    
    class_df = pd.DataFrame(model.classes_).reset_index()
    
    # data frame 생성
    for i in range(len(model.classes_)):
        class_df.columns = [i, 'rank_'+str(i)]
        test_class = pd.merge(test_class, class_df, on=i, how='left')

    # rank columns    
    rank_columns = ['rank_0',  'rank_1',  'rank_2',  'rank_3',  'rank_4',
                    'rank_5',  'rank_6',  'rank_7',  'rank_8',  'rank_9', 
                    'rank_10', 'rank_11', 'rank_12', 'rank_13', 'rank_14', 
                    'rank_15', 'rank_16', 'rank_17', 'rank_18']

    #dataframe to csv
    test_class_csv = test_class[rank_columns].to_csv(header=False, index=False)
    test_class_csv = test_class_csv.splitlines()
    
    return test_class_csv#test_class_csv[0], '...\n \n ', test_class_csv[-1]

In [27]:
# predict rank
rank_class = predict_rank(test_X) # test score 0.87846

In [575]:
# predict 결과 확인
rank_class[:2]

['연합뉴스,아시아경제,뉴시스,파이낸셜뉴스,JTBC,채널A,오마이뉴스,연합뉴스TV,서울신문,매일경제,동아일보,국민일보,경향신문,YTN,TV조선,MBN,KBS 뉴스,노컷뉴스,헤럴드경제',
 '뉴시스,연합뉴스,MBN,JTBC,채널A,오마이뉴스,연합뉴스TV,아시아경제,서울신문,매일경제,동아일보,노컷뉴스,국민일보,경향신문,YTN,TV조선,KBS 뉴스,파이낸셜뉴스,헤럴드경제']

In [576]:
# write csv
output_file = 'submission.txt'
with open(output_file, 'w') as fp:
    for row in rank_class:
        fp.write(row)
        fp.write('\n')

#### MRR

In [29]:
def mrr(rank_class, test_out_data):

    pred = []
    for i in range(len(rank_class)):
        pred.append(rank_class[i].split(','))
    
    position = np.zeros([len(rank_class),1])
    pred = np.array(pred)
    test_true = np.array(test_out_data)
    
    for i in range(len(rank_class)):
        position[i] = np.where(pred[i]==test_true[i])[0]+1
    
    return ((1/position).sum())/len(rank_class)

In [34]:
mrr(rank_class, test_out_data) #logistic C = 10 featurelist

0.86249111074111073

In [34]:
mrr(rank_class, test_out_data) #logistic C = 10000 true true stw

0.93987501570210552

In [45]:
#grid search logistic C 2

alpha=[0.01, 0.1, 1, 15, 25, 45]
for i,val in enumerate(alpha):
    model = LogisticRegression(C=val).fit(X,y)
    rank_class = predict_rank(test_X)
    print(val, mrr(rank_class, test_df.writing))

0.01 0.480534784582
0.1 0.650051231219
1 0.783010539492
15 0.870699469666
25 0.87711188102
45 0.880247206497


In [64]:
import sys
sys.stdout.flush()

In [ ]:
#grid search logistic C 5

alpha=[9900, 10100, 10200, 10300]
for i,val in enumerate(alpha):
    model = LogisticRegression(C=val).fit(X,y)
    rank_class = predict_rank(test_X)
    print(val, mrr(rank_class, test_df.writing))

9900 0.894274588774
10100 0.893767222991
10200 0.894535876252


In [70]:
#grid search logistic C 5

alpha=[11000, 12000, 13000]
for i,val in enumerate(alpha):
    model = LogisticRegression(C=val).fit(X,y)
    rank_class = predict_rank(test_X)
    print(val, mrr(rank_class, test_df.writing))

11000 0.894079127753
12000 0.89368093798
13000 0.894096626612


In [66]:
#grid search logistic C 4

alpha=[20000, 40000, 60000]
for i,val in enumerate(alpha):
    model = LogisticRegression(C=val).fit(X,y)
    rank_class = predict_rank(test_X)
    print(val, mrr(rank_class, test_df.writing))

20000 0.893717458649
40000 0.893469522018
60000 0.892794708033


In [65]:
#grid search logistic C 3

alpha=[2000, 10000, 100000]
for i,val in enumerate(alpha):
    model = LogisticRegression(C=val).fit(X,y)
    rank_class = predict_rank(test_X)
    print(val, mrr(rank_class, test_df.writing))

2000 0.892520188544
10000 0.894128997945
100000 0.893223218938


In [ ]:
# 1000 -> 0.891

In [46]:
#grid search logistic C 2

alpha=[55, 65, 75, 85, 95, 105]
for i,val in enumerate(alpha):
    model = LogisticRegression(C=val).fit(X,y)
    rank_class = predict_rank(test_X)
    print(val, mrr(rank_class, test_df.writing))

55 0.881877988061
65 0.882586373503
75 0.884111527362
85 0.884764261048
95 0.885041231608
105 0.885410111802


#### Predict grading_data

In [577]:
grading_df['text'][20:30]

20    취재진 질문에 답하는 조인근 전 비서관\n    (서울=<언론사명>)  = 조인근 ...
21    [<언론사명>]사진=<언론사명> 캡처 ‘최순실 게이트’의 핵심 증거 중 하나로 꼽히...
22    최경환 의원(왼쪽)과 안종범 수석(오른쪽)\n기재위, 청문회 실시계획서 채택 불발…...
23    김 전 차관 오전 10시, 최씨 오후 2시 출석 예정 삼성그룹의 최씨 일가 특혜 지...
24    [<언론사명>]김종인 공격 국민의당 되레↓ 쟁점 법안 촉구 땐 13%대 반등 여론조...
25     = 정부가 북한의 gps 전파 교란 도발과 관련해 유엔 안전보장이사회와 관련 국제...
26    북한이 연일 미국과 한반도 주변 국가간 갈등을 부각하며 미국의 "쇠퇴"를 주장하고 ...
27    임명장 수여식·공식발표 없는 취임식 강행은 경찰청 출범 25년 만에 ][기사보강: ...
28    여론조사 전문기관 리얼미터(대표 이택수)의 2016년 1월 4주차(25~29일) 여...
29    총선 주요 공약 강봉균 공동선대위원장이 직접  = 새누리당은 28일 공천자대회와 선...
Name: text, dtype: object

In [31]:
#grading to vec
X_grading = grading_df['text']
X_grading = testX_to_vec(X_grading)

In [52]:
# predict rank
grading_rank_class = predict_rank(X_grading) # test score 0.87846

In [44]:
# predict 결과 확인
grading_rank_class[19]

'매일경제,뉴시스,연합뉴스,MBN,파이낸셜뉴스,국민일보,오마이뉴스,아시아경제,KBS 뉴스,JTBC,연합뉴스TV,노컷뉴스,채널A,TV조선,YTN,동아일보,헤럴드경제,서울신문,경향신문'

In [53]:
# write csv
output_file = 'submission.txt'
with open(output_file, 'w') as fp:
    for row in grading_rank_class:
        fp.write(row)
        fp.write('\n')

#### Submission

In [44]:
ec.upload()

이전에 업로드 된 파일 삭제 중...

두 종류의 파일들을 제출해야 합니다.
 - submission.txt : grading.input.json 에 해당하는 출력값
 - submission.ipynb 혹은 submission.py : submission.txt를 재현가능한 소스 코드
제출할 파일들은 comma (,) 로 나누어 써서 제출할 수 있습니다.
제출 예제) submission.ipynb,submission.txt,additional_data.json

> submission.txt,submission.ipynb
submission.txt 체크 중...
submission.txt: 2016000 bytes
submission.ipynb 체크 중...
submission.ipynb: 5856 bytes
업로드 중: submission.txt
업로드 중: submission.ipynb
업로드가 완료되었습니다.
submit() 으로 점수를 확인할 수 있습니다.


업로드가 완료되면 제출을 시도해봅니다.

In [45]:
ec.submit()

submission 데이터 개수 = 9000
답안 데이터 개수 = 9000
MRR: 0.18645
Score: 18 pts.
18
Score: 18
